In [78]:
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
from typing import Text, Union
import copy
import math
from qlib.utils import get_or_create_path
from qlib.log import get_module_logger

import torch
import torch.nn as nn
import torch.optim as optim

from qlib.model.base import Model
from qlib.data.dataset import DatasetH
from qlib.data.dataset.handler import DataHandlerLP
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # [T, N, F]
        return x + self.pe[: x.size(0), :]


class Transformer(nn.Module):
    def __init__(self, d_feat=6, d_model=8, nhead=4, num_layers=2, dropout=0.5, device=None):
        super(Transformer, self).__init__()
        self.feature_layer = nn.Linear(d_feat, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder_layer = nn.Linear(d_model, 1)
        self.device = device
        self.d_feat = d_feat

    def forward(self, src):
        # src [N, F*T] --> [N, T, F]
        src = src.reshape(len(src), self.d_feat, -1).permute(0, 2, 1)
        src = self.feature_layer(src)

        # src [N, T, F] --> [T, N, F], [60, 512, 8]
        src = src.transpose(1, 0)  # not batch first

        mask = None

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, mask)  # [60, 512, 8]

        # [T, N, F] --> [N, T*F]
        output = self.decoder_layer(output.transpose(1, 0)[:, -1, :])  # [512, 1]

        return output.squeeze()
class LSTMModel(nn.Module):
    def __init__(self, d_feat=6, hidden_size=64, num_layers=2, dropout=0.0):
        super().__init__()

        self.rnn = nn.LSTM(
            input_size=d_feat,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc_out = nn.Linear(hidden_size, 1)

        self.d_feat = d_feat

    def forward(self, x):
        # x: [N, F*T]
        x = x.reshape(len(x), self.d_feat, -1)  # [N, F, T]
        x = x.permute(0, 2, 1)  # [N, T, F]
        out, _ = self.rnn(x)
        return self.fc_out(out[:, -1, :]).squeeze()
class GRUModel(nn.Module):
    def __init__(self, d_feat=6, hidden_size=64, num_layers=2, dropout=0.0):
        super().__init__()

        self.rnn = nn.GRU(
            input_size=d_feat,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc_out = nn.Linear(hidden_size, 1)

        self.d_feat = d_feat

    def forward(self, x):
        # x: [N, F*T]
        x = x.reshape(len(x), self.d_feat, -1)  # [N, F, T]
        x = x.permute(0, 2, 1)  # [N, T, F]
        out, _ = self.rnn(x)
        return self.fc_out(out[:, -1, :]).squeeze()
class ALSTMModel(nn.Module):
    def __init__(self, d_feat=6, hidden_size=64, num_layers=2, dropout=0.0, rnn_type="GRU"):
        super().__init__()
        self.hid_size = hidden_size
        self.input_size = d_feat
        self.dropout = dropout
        self.rnn_type = rnn_type
        self.rnn_layer = num_layers
        self._build_model()

    def _build_model(self):
        try:
            klass = getattr(nn, self.rnn_type.upper())
        except Exception as e:
            raise ValueError("unknown rnn_type `%s`" % self.rnn_type) from e
        self.net = nn.Sequential()
        self.net.add_module("fc_in", nn.Linear(in_features=self.input_size, out_features=self.hid_size))
        self.net.add_module("act", nn.Tanh())
        self.rnn = klass(
            input_size=self.hid_size,
            hidden_size=self.hid_size,
            num_layers=self.rnn_layer,
            batch_first=True,
            dropout=self.dropout,
        )
        self.fc_out = nn.Linear(in_features=self.hid_size * 2, out_features=1)
        self.att_net = nn.Sequential()
        self.att_net.add_module(
            "att_fc_in",
            nn.Linear(in_features=self.hid_size, out_features=int(self.hid_size / 2)),
        )
        self.att_net.add_module("att_dropout", torch.nn.Dropout(self.dropout))
        self.att_net.add_module("att_act", nn.Tanh())
        self.att_net.add_module(
            "att_fc_out",
            nn.Linear(in_features=int(self.hid_size / 2), out_features=1, bias=False),
        )
        self.att_net.add_module("att_softmax", nn.Softmax(dim=1))

    def forward(self, inputs):
        # inputs: [batch_size, input_size*input_day]
        inputs = inputs.view(len(inputs), self.input_size, -1)
        inputs = inputs.permute(0, 2, 1)  # [batch, input_size, seq_len] -> [batch, seq_len, input_size]
        rnn_out, _ = self.rnn(self.net(inputs))  # [batch, seq_len, num_directions * hidden_size]
        attention_score = self.att_net(rnn_out)  # [batch, seq_len, 1]
        out_att = torch.mul(rnn_out, attention_score)
        out_att = torch.sum(out_att, dim=1)
        out = self.fc_out(
            torch.cat((rnn_out[:, -1, :], out_att), dim=1)
        )  # [batch, seq_len, num_directions * hidden_size] -> [batch, 1]
        return out[..., 0]

In [79]:
class Trader:
    def __init__(self,factors,d_feat,batch_size,lr=0.001,optimizer="adam"):
        import random
        self.inputfactorsnums=random.sample(range(len(factors)), 6)#factors=['KLEN', 'KLOW', 'ROC60', 'STD5', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR60', 'RESI5', 'RESI10', 'CORR5', 'CORR10', 'CORR20', 'CORR60', 'CORD5','CORD10', 'CORD60', 'VSTD5', 'WVMA5', 'WVMA60']
        self.inputfactors=[]
        for i in self.inputfactorsnums:
            self.inputfactors.append(factors[i])
        self.nmodel= random.randint(0, 3)
        self.train_loss=0
        self.stop_steps=0
        self.train_score = 0
        self.best_score=-np.inf
        self.train_loss=0
        self.train_score =-np.inf
        self.val_loss=0
        self.val_score=-np.inf
        self.predicted_data = np.array([])
        self.actual_data=np.array([])
        self.overall_accuracy=-1
        self.lr=lr
        self.perstockacc= np.array([])
        if self.nmodel==0:
            self.Model =Transformer(d_feat=d_feat,num_layers=2)
        elif self.nmodel == 1:
            self.Model = ALSTMModel(d_feat,num_layers =  2)
        elif self.nmodel== 2:
            self.Model =GRUModel(d_feat =d_feat, num_layers = 2)
        elif self.nmodel == 3:
            self.Model = LSTMModel(d_feat =d_feat, num_layers = 2)
        if optimizer.lower() == "adam":
            self.train_optimizer = optim.Adam(self.Model.parameters(), lr=self.lr)
        elif optimizer.lower() == "gd":
            self.train_optimizer = optim.SGD(self.Model.parameters(), lr=self.lr)
    def acc(self,lookback,nstcok):
        mae_scores = []
        from sklearn.metrics import mean_absolute_error
        for i in range(10):
            actual_values = self.actual_data.reshape(lookback,nstcok)[:,i]
            predicted_values = self.predicted_data.reshape(lookback,nstcok)[:,i]
            mae = mean_absolute_error(actual_values, predicted_values)
            mae_scores.append(mae)
        overall_accuracy = np.mean(mae_scores)
        self.perstockacc=mae_scores
        self.overall_accuracy= overall_accuracy
    def mse(self, pred, label):
        loss = (pred - label) ** 2
        return torch.mean(loss)
    def train_item(self, x,y):
        self.Model.train()
        pred = self.Model(x)
        loss = self.mse(pred,torch.tensor(y,requires_grad=True))
        self.train_optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(self.Model.parameters(), 3.0)
        self.train_optimizer.step()
        return pred.detach().numpy()

In [89]:
class CompanyModel(nn.Module):
    def __init__(self,models):
        super(CompanyModel, self).__init__()#name='FactorVAE')
        self.models =models
        self.fc_out = nn.Linear(64, 1)
    def forward(self, x_train,y_train,train=False):

        # x.shape == (Ns, T, C)
        # if training:
        if train==True:
            scores=[]
            for (i,model) in enumerate( self.models):
                yi_pred=c.traders[i].train_item(torch.from_numpy(x_train[c.traders[i].inputfactors].values).float(),y_train)
                #yi_pred=model(torch.from_numpy(x_train[c.traders[i].inputfactors].values).float()).detach().numpy()
                if(len(c.traders[i].predicted_data)<c.lookback*len(yi_pred)):
                    c.traders[i].predicted_data=np.append(c.traders[i].predicted_data,yi_pred)
                    c.traders[i].actual_data=np.append(c.traders[i].predicted_data,y_train)
                    scores.append(0)
                else:
                    c.traders[i].predicted_data=np.append(c.traders[i].predicted_data,yi_pred)
                    c.traders[i].actual_data=np.append(c.traders[i].predicted_data,y_train)
                    c.traders[i].predicted_data = c.traders[i].predicted_data[-200:]
                    c.traders[i].actual_data=c.traders[i].actual_data[-200:]
                    c.traders[i].acc(c.lookback,len(yi_pred))
                    scores.append(c.traders[i].overall_accuracy)

            sorted_list = sorted(scores)
            smallest_indices = [scores.index(sorted_list[i]) for i in range(c.aggnum)]
            ans=[c.traders[i].predicted_data[-len(yi_pred):] for i in smallest_indices]
            return sum(ans)/c.aggnum
        else:
            scores=[]
            for (i,model) in enumerate( self.models):
                yi_pred=model(torch.from_numpy(x_train[c.traders[i].inputfactors].values).float()).detach().numpy()
                scores.append(c.traders[i].overall_accuracy)
            sorted_list = sorted(scores)
            smallest_indices = [scores.index(sorted_list[i]) for i in range(c.aggnum)]
            ans=[c.traders[i].predicted_data[-len(yi_pred):] for i in smallest_indices]
            for (i,model) in enumerate( self.models):
                c.traders[i].predicted_data=np.append(c.traders[i].predicted_data,yi_pred)
                c.traders[i].actual_data=np.append(c.traders[i].predicted_data,y_train)
                c.traders[i].predicted_data = c.traders[i].predicted_data[-200:]
                c.traders[i].actual_data=c.traders[i].actual_data[-200:]
                c.traders[i].acc(c.lookback,len(yi_pred))
            return sum(ans)/c.aggnum


class Company:
    def __init__(self,
                 factors:[],
                 n_traders: int,
                 d_feat: int,
                 lookback=20,
                 batch_size=1,
                 optimizer="adam",
                 aggnum=3,
                 lr=0.001,
                 seed = 1
                 ):
        self.aggnum=aggnum
        self.factors=factors
        self.lookback=lookback
        self.n_traders = n_traders
        self.d_feat =  d_feat
        self.batch_size =  batch_size
        self.lr = lr
        self.logger = get_module_logger("TransformerModel")
        self.traders= [Trader(self.factors,self.d_feat,self.batch_size) for n in range(n_traders)]
        modells=[i.Model for i in self.traders]
        self.Model=CompanyModel(modells)
        self.device = torch.device("cuda:%d" % 0if torch.cuda.is_available() and 0 >= 0 else "cpu")
        if optimizer.lower() == "adam":
            self.train_optimizer = optim.Adam(self.Model.parameters(), lr=self.lr)
        elif optimizer.lower() == "gd":
            self.train_optimizer = optim.SGD(self.Model.parameters(), lr=self.lr)

    def mse(self, pred, label):
        loss = (pred - label) ** 2
        return torch.mean(loss)

    def train_epoch(self, h,nstock):
        self.Model.train()
        l=int(len(h.fetch(col_set="label").values)/nstock)
        indices = np.arange(l)
        np.random.shuffle(indices)
        losses=[]
        for i in range(l):
            x=h.fetch(col_set="feature").iloc[10*i:10*i+10]
            y=h.fetch(col_set="label").iloc[10*i:10*i+10].values
            pred = self.Model(x,y,True)
            loss = self.mse(torch.tensor(pred,requires_grad=True),torch.tensor(y,requires_grad=True))
            self.train_optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(self.Model.parameters(), 3.0)
            self.train_optimizer.step()
            losses.append(float(loss))
        print("trainloss:"+str(sum(losses) / len(losses)))
    def test_epoch(self, h,nstock):
        l=int(len(h.fetch(col_set="label").values)/nstock)
        indices = np.arange(l)
        losses=[]
        rankic=[]
        for i in range(l):
            x=h.fetch(col_set="feature").iloc[10*i:10*i+10]
            y=h.fetch(col_set="label").iloc[10*i:10*i+10].values
            pred = self.Model(x,y)
            loss = self.mse(torch.tensor(pred,requires_grad=True),torch.tensor(y,requires_grad=True))
            losses.append(float(loss))

        print("testloss:"+str(sum(losses) / len(losses)))


In [90]:
import qlib
from qlib.config import REG_CN
from qlib.data.dataset.loader import QlibDataLoader
from qlib.data.dataset.processor import ZScoreNorm, Fillna
from qlib.contrib.data.handler import Alpha158

qlib.init()

data_handler_config = {
    "start_time": "2017-01-01",
    "end_time": "2018-11-10",
    "instruments": ['SH000923','SH600015','SH600019','SH600028','SH600030','SH600036','SH600048','SH600050','SH600104','SH600519'],
     "infer_processors":[ZScoreNorm(fit_start_time='2017-01-01', fit_end_time="2018-11-10"), Fillna()],
}
h = Alpha158(**data_handler_config)


[94573:MainThread](2023-04-03 19:33:12,348) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[94573:MainThread](2023-04-03 19:33:12,350) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[94573:MainThread](2023-04-03 19:33:12,351) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/linweiqiang/.qlib/qlib_data/cn_data')}
[94573:MainThread](2023-04-03 19:33:21,451) INFO - qlib.timer - [log.py:117] - Time cost: 9.099s | Loading data Done
/Users/linweiqiang/opt/anaconda3/lib/python3.9/site-packages/pyqlib-0.8.5.99-py3.9-macosx-10.9-x86_64.egg/qlib/data/dataset/processor.py:242: RuntimeWarning: Mean of empty slice
  self.mean_train = np.nanmean(df[cols].values, axis=0)
/Users/linweiqiang/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
[945

In [91]:
data_handler_config = {
    "start_time": "2018-11-11",
    "end_time": "2018-12-10",
    "instruments": ['SH000923','SH600015','SH600019','SH600028','SH600030','SH600036','SH600048','SH600050','SH600104','SH600519'],
     "infer_processors":[ZScoreNorm(fit_start_time= "2018-11-11", fit_end_time="2018-12-10"), Fillna()],
}
h2= Alpha158(**data_handler_config)

[94573:MainThread](2023-04-03 19:33:30,258) INFO - qlib.timer - [log.py:117] - Time cost: 8.317s | Loading data Done
/Users/linweiqiang/opt/anaconda3/lib/python3.9/site-packages/pyqlib-0.8.5.99-py3.9-macosx-10.9-x86_64.egg/qlib/data/dataset/processor.py:242: RuntimeWarning: Mean of empty slice
  self.mean_train = np.nanmean(df[cols].values, axis=0)
/Users/linweiqiang/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
[94573:MainThread](2023-04-03 19:33:30,286) INFO - qlib.timer - [log.py:117] - Time cost: 0.026s | ZScoreNorm Done
[94573:MainThread](2023-04-03 19:33:30,288) INFO - qlib.timer - [log.py:117] - Time cost: 0.002s | Fillna Done
[94573:MainThread](2023-04-03 19:33:30,291) INFO - qlib.timer - [log.py:117] - Time cost: 0.002s | DropnaLabel Done
[94573:MainThread](2023-04-03 19:33:30,316) INFO - qlib.timer - [log.py:117] - Time cost: 0.024s | CS

In [92]:
c=Company(h.get_cols()[0:158],20,6,20,1)

In [93]:
for i in range(5000):
    c.train_epoch(h,10)
    c.test_epoch(h2,10)

trainloss:0.9783354238846674
testloss:1.069811001252227
trainloss:0.9718581991213814
testloss:1.0332065823278809
trainloss:0.9714907155999852
testloss:1.0436734918328916
trainloss:0.9714499230822994
testloss:1.0388451660062732
trainloss:0.9714825566447042
testloss:1.0389929604532215
trainloss:0.9710013990401198
testloss:1.0431691949955364
trainloss:0.9707470149457661
testloss:1.0432182162212245
trainloss:0.9708983006196886
testloss:1.045039912732901
trainloss:0.9707088618531623
testloss:1.0368773081577443
trainloss:0.970724549969598
testloss:1.0432713643009222
trainloss:0.9706960021617184
testloss:1.0427402771746104
trainloss:0.9708352251405841
testloss:1.0449235157584906
trainloss:0.9708239143556926
testloss:1.0428149743169628
trainloss:0.9706458291601165
testloss:1.044329292853873
trainloss:0.9706213266952444
testloss:1.0395804561626842
trainloss:0.9701536585458793
testloss:1.0446493318471881
trainloss:0.9704813976933913
testloss:1.0391007735660474
trainloss:0.9703203772824748
testlo